In [4]:
import numpy as np
import rasterio
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Define paths to Landsat Geotiff and LULC map
landsat_path = "C:/Users/mrg22-020914479/Downloads/lulc.tif"
lulc_path = "C:/Users/mrg22-020914479/Downloads/TestDL_2022.tif"

# Open Landsat Geotiff file
with rasterio.open(landsat_path) as landsat_ds:
    # Read all bands
    landsat_data = np.stack([landsat_ds.read(band_idx) for band_idx in range(1, landsat_ds.count + 1)], axis=-1)
    landsat_data[np.isnan(landsat_data)] = 0
# Open LULC map Geotiff file
with rasterio.open(lulc_path) as lulc_ds:
    lulc_data = lulc_ds.read(1)  # Assuming LULC is a single-band GeoTIFF
    lulc_data[np.isnan(lulc_data)] = 0

# Define the number of classes in the LULC map
num_classes = len(np.unique(lulc_data))

# Flatten the data for training
landsat_data_flat = landsat_data.reshape(-1, landsat_data.shape[-1])
lulc_data_flat = lulc_data.flatten()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(landsat_data_flat, lulc_data_flat, test_size=0.3, random_state=42)

# Train a simple Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# You can now use this trained classifier to make predictions on new Landsat data.


Accuracy: 0.20


In [5]:
# Import necessary libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import compute_class_weight

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

# Assuming landsat_data is a 3D array with shape (num_bands, height, width)
# and lulc_data is a 2D array with shape (height, width)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    landsat_data.transpose(1, 2, 0),  # Reshape Landsat data to (height, width, num_bands)
    lulc_data,
    test_size=0.2,  # Adjust the test size as needed
    random_state=42
)

# Define a simple CNN model
model = models.Sequential([
    layers.Input(shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3])),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')  # Adjust num_classes based on your LULC classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')

# You can save the trained model for future use
model.save('landsat_lulc_classification_model.h5')


In [9]:
# Import necessary libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import compute_class_weight

# Load your Landsat images and corresponding LULC maps here.
# You'll need to prepare your data as NumPy arrays or load them using an appropriate library.
# Make sure to resize or preprocess the images and maps as needed.

# Define your model architecture
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(landsat_data.shape)),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss=SparseCategoricalCrossentropy(),
              metrics=[SparseCategoricalAccuracy()])

# Split your data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(landsat_data, lulc_data, test_size=0.2, random_state=42)

# Encode LULC map labels (e.g., using LabelEncoder)
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

# Compute class weights for imbalanced datasets (optional)
class_weights = compute_class_weight('balanced', np.unique(y_train_encoded), y_train_encoded)

# Train the model
history = model.fit(X_train, y_train_encoded, 
                    epochs=num_epochs, 
                    batch_size=batch_size, 
                    validation_data=(X_val, y_val_encoded),
                    class_weight=class_weights if use_class_weights else None)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test_encoded)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

ValueError: y should be a 1d array, got an array of shape (612, 701) instead.